# Angular Velocity: Intuitive 3D Animations (No glTF airplane dependency)
Your error came from airplane meshes being **glTF 1.0** (trimesh supports glTF 2.0 only).

This notebook avoids that by using:
- a **procedural airplane mesh** (fuselage + wings + tail)
- a **real robot mesh** from MoveIt if `git` works; otherwise a procedural gripper mesh

Each demo shows:
- angular velocity vector \(\omega\) (labeled as **space frame** or **body frame**)
- space axes (fixed) and body axes (rotating)
- a marked point (wing tip / fingertip) with tangential velocity \(v=\omega\times r\)

**Coordinate Frames:**
- **Space frame**: Fixed inertial frame (shown as gray axes)
- **Body frame**: Frame attached to the rotating object (shown as colored axes)
- **Angular velocity \(\omega\)**: 
  - When computed from \(R(t)\) using \(\omega^\wedge = \dot{R} R^T\), we get \(\omega\) in the **space frame**
  - Body-frame angular velocity: \(\omega_{body} = R^T \omega_{space}\)

**Saved Animations:**
All animations are saved as interactive HTML files in the `assets_meshes/` folder for later use.


In [ ]:
import os, glob, math, subprocess, shutil
import numpy as np

import plotly.graph_objects as go

import trimesh

np.set_printoptions(precision=4, suppress=True)

## Helpers

In [ ]:
def unit(v, eps=1e-12):
    v = np.asarray(v, dtype=float).reshape(-1)
    n = np.linalg.norm(v)
    return v if n < eps else v/n

def skew(w):
    w1,w2,w3 = np.asarray(w, dtype=float).reshape(3,)
    return np.array([[0, -w3, w2],
                     [w3, 0, -w1],
                     [-w2, w1, 0]], dtype=float)

def rodrigues(axis_or_omega, theta):
    axis = unit(axis_or_omega)
    K = skew(axis)
    I = np.eye(3)
    return I + math.sin(theta)*K + (1-math.cos(theta))*(K@K)

def apply_R(R, V):
    return (R @ np.asarray(V).T).T

def mesh_to_plotly(mesh: trimesh.Trimesh, name="mesh", opacity=1.0):
    v = mesh.vertices
    f = mesh.faces
    return go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        name=name,
        opacity=opacity,
        flatshading=True,
        showscale=False,
    )

def axes_traces(R=np.eye(3), origin=(0,0,0), L=1.0):
    o = np.array(origin, float)
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    traces = []
    for vec in [ex, ey, ez]:
        p = o + L*vec
        traces.append(go.Scatter3d(
            x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
            mode="lines",
            line=dict(width=7),
            showlegend=False
        ))
    return traces

def arrow_trace(vec, origin=(0,0,0), name="arrow", scale=1.0, width=10):
    o = np.array(origin, float)
    v = np.array(vec, float)*scale
    p = o + v
    return go.Scatter3d(
        x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
        mode="lines+markers",
        name=name,
        marker=dict(size=3),
        line=dict(width=width),
        showlegend=False
    )

def set_scene(fig, lim=2.0, title=""):
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(range=[-lim, lim]),
            yaxis=dict(range=[-lim, lim]),
            zaxis=dict(range=[-lim, lim]),
            aspectmode="cube"
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig

def save_animation(fig, filename, assets_dir="assets_meshes", format="html"):
    """
    Save Plotly animation to assets folder.
    
    Args:
        fig: Plotly figure with animation frames
        filename: Output filename (without extension)
        assets_dir: Directory to save to (default: assets_meshes)
        format: "html" (interactive) or "png" (frames for video conversion)
    """
    os.makedirs(assets_dir, exist_ok=True)
    
    if format == "html":
        filepath = os.path.join(assets_dir, f"{filename}.html")
        fig.write_html(filepath)
        print(f"✓ Saved interactive animation: {filepath}")
        return filepath
    elif format == "png":
        # Export frames as PNG (requires kaleido)
        frames_dir = os.path.join(assets_dir, f"{filename}_frames")
        os.makedirs(frames_dir, exist_ok=True)
        try:
            import plotly.io as pio
            for i, frame in enumerate(fig.frames):
                # Create a figure with just this frame's data
                frame_fig = go.Figure(data=frame.data)
                frame_fig.update_layout(fig.layout)
                frame_path = os.path.join(frames_dir, f"frame_{i:04d}.png")
                frame_fig.write_image(frame_path, width=1920, height=1080)
            print(f"✓ Saved {len(fig.frames)} frames to: {frames_dir}")
            print(f"  To create video: ffmpeg -r 20 -i {frames_dir}/frame_%04d.png -vcodec libx264 {assets_dir}/{filename}.mp4")
            return frames_dir
        except Exception as e:
            print(f"⚠ PNG export failed (install kaleido: pip install kaleido): {e}")
            print(f"  Falling back to HTML export...")
            return save_animation(fig, filename, assets_dir, format="html")
    else:
        raise ValueError(f"Unknown format: {format}")

def normalize_mesh(mesh: trimesh.Trimesh, target=1.0):
    if mesh is None: 
        return None
    mesh = mesh.copy()
    c = mesh.bounding_box.centroid
    mesh.apply_translation(-c)
    ext = mesh.extents
    s = (2*target) / (np.max(ext) + 1e-12)
    mesh.apply_scale(s)
    return mesh

## Build a simple airplane mesh (procedural)

In [ ]:
def make_airplane_mesh():
    # Fuselage along +x
    fus = trimesh.creation.cylinder(radius=0.12, height=2.0, sections=32)
    fus.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))

    nose = trimesh.creation.icosphere(subdivisions=2, radius=0.14)
    nose.apply_translation([1.05, 0.0, 0.0])

    wing = trimesh.creation.box(extents=[0.25, 1.3, 0.03])

    tail = trimesh.creation.box(extents=[0.18, 0.55, 0.02])
    tail.apply_translation([-0.85, 0.0, 0.02])

    fin = trimesh.creation.box(extents=[0.12, 0.04, 0.25])
    fin.apply_translation([-0.9, 0.0, 0.15])

    plane = trimesh.util.concatenate([fus, nose, wing, tail, fin])
    return normalize_mesh(plane, target=1.0)

air_mesh = make_airplane_mesh()
print("air mesh:", len(air_mesh.vertices), len(air_mesh.faces))

## Load a real robot mesh (MoveIt) if possible; otherwise fallback

In [ ]:
ASSETS_DIR = "assets_meshes"
os.makedirs(ASSETS_DIR, exist_ok=True)

# --- helpers (keep compatible across trimesh versions) ---

def git_clone_if_needed(url, dst, depth=1):
    if os.path.exists(dst) and os.path.isdir(dst) and os.listdir(dst):
        return True
    try:
        if os.path.exists(dst):
            shutil.rmtree(dst)
        subprocess.check_call(["git", "clone", "--depth", str(depth), url, dst])
        return True
    except Exception as e:
        print("git clone failed:", e)
        return False


def load_mesh_any(path):
    if path is None:
        return None
    mesh_or_scene = trimesh.load(path, force="scene")
    if isinstance(mesh_or_scene, trimesh.Scene):
        geoms = [g for g in mesh_or_scene.geometry.values() if hasattr(g, 'vertices')]
        if not geoms:
            return None
        mesh = trimesh.util.concatenate(geoms)
    else:
        mesh = mesh_or_scene

    mesh = mesh.copy()
    if hasattr(mesh, 'remove_unreferenced_vertices'):
        mesh.remove_unreferenced_vertices()
    if hasattr(mesh, 'remove_duplicate_faces'):
        mesh.remove_duplicate_faces()
    if hasattr(mesh, 'remove_degenerate_faces'):
        mesh.remove_degenerate_faces()
    return mesh


def maybe_decimate(mesh: trimesh.Trimesh, target_faces=2500):
    """Try to decimate for Plotly (optional). Requires `fast-simplification`."""
    if mesh is None:
        return None
    try:
        import fast_simplification  # noqa: F401
    except Exception:
        return mesh

    try:
        if hasattr(mesh, 'simplify_quadratic_decimation') and len(mesh.faces) > target_faces:
            return mesh.simplify_quadratic_decimation(face_count=int(target_faces))
    except Exception:
        pass
    return mesh


def make_simple_gripper():
    palm = trimesh.creation.box(extents=[0.35, 0.18, 0.12])
    finger1 = trimesh.creation.box(extents=[0.35, 0.05, 0.06])
    finger2 = trimesh.creation.box(extents=[0.35, 0.05, 0.06])
    finger1.apply_translation([0.15, 0.10, 0.00])
    finger2.apply_translation([0.15, -0.10, 0.00])
    g = trimesh.util.concatenate([palm, finger1, finger2])
    return normalize_mesh(g, target=0.9)


print("Attempting to load an industrial gripper/hand mesh (prefer Robotiq 2F, then Franka, then MoveIt)...")

# Track all successfully loaded meshes
loaded_meshes = []  # List of (mesh, name, source) tuples
failed_attempts = []  # List of (source_name, reason) tuples
robot_mesh = None
robot_path = None

# 1) Robotiq (nice-looking industrial gripper)
robotiq_repo = os.path.join(ASSETS_DIR, "robotiq")
print(f"\n[1/4] Trying Robotiq repo: {robotiq_repo}")
try:
    ok_robotiq = git_clone_if_needed("https://github.com/ros-industrial/robotiq.git", robotiq_repo)
    if not ok_robotiq:
        failed_attempts.append(("Robotiq", "Git clone failed or repository not accessible"))
        print("  ❌ Failed: Git clone failed or repository not accessible")
except Exception as e:
    ok_robotiq = False
    failed_attempts.append(("Robotiq", f"Git clone error: {str(e)[:100]}"))
    print(f"  ❌ Failed: Git clone error - {str(e)[:100]}")

if ok_robotiq:
    # Look for visual meshes first (often .stl), then anything.
    robotiq_candidates = []
    for pat in ["**/*visual*/*.stl", "**/*visual*/*.dae", "**/*/*.stl", "**/*/*.dae"]:
        robotiq_candidates += glob.glob(os.path.join(robotiq_repo, pat), recursive=True)

    # Prefer 2f grippers
    def score_robotiq(p):
        s = 0
        pl = p.lower()
        if "2f" in pl:
            s += 2000
        if "85" in pl or "140" in pl:
            s += 200
        if "visual" in pl:
            s += 500
        if pl.endswith('.dae'):
            s += 50
        try:
            s += min(os.path.getsize(p)//1000, 400)
        except Exception:
            pass
        return s

    robotiq_candidates = sorted(list(set(robotiq_candidates)), key=score_robotiq, reverse=True)

    # Many Robotiq meshes are multiple parts. We'll try to load a handful and concatenate.
    parts = []
    part_names = []
    failed_files = []
    
    if not robotiq_candidates:
        failed_attempts.append(("Robotiq", "No mesh files found in repository"))
        print("  ❌ Failed: No mesh files found in repository")
    else:
        print(f"  Found {len(robotiq_candidates)} candidate files, trying to load...")
        for p in robotiq_candidates[:12]:
            try:
                m = load_mesh_any(p)
                if m is None or len(m.vertices) == 0:
                    failed_files.append((os.path.basename(p), "Empty mesh or no vertices"))
                    continue
                # Keep only reasonably-sized parts to avoid grabbing huge scenes
                parts.append(m)
                part_names.append(os.path.relpath(p, robotiq_repo))
                if len(parts) >= 6:
                    break
            except Exception as e:
                failed_files.append((os.path.basename(p), f"Load error: {str(e)[:80]}"))
                continue

        if parts:
            robot_mesh = trimesh.util.concatenate(parts)
            robot_mesh = normalize_mesh(robot_mesh, target=0.9)
            robot_mesh = maybe_decimate(robot_mesh, target_faces=3000)
            robot_path = f"Robotiq ({len(parts)} parts): " + ", ".join(part_names[:3]) + (" ..." if len(part_names) > 3 else "")
            print(f"  ✓ Successfully loaded {len(parts)} parts")
            loaded_meshes.append((robot_mesh.copy(), "Robotiq Gripper", robot_path))
        else:
            reason = f"No valid mesh parts loaded. {len(failed_files)} files failed"
            if failed_files:
                reason += f" (e.g., {failed_files[0][0]}: {failed_files[0][1]})"
            failed_attempts.append(("Robotiq", reason))
            print(f"  ❌ Failed: {reason}")

# 2) Franka (pretty hand, needs pycollada for .dae)
if robot_mesh is None:
    print(f"\n[2/4] Trying Franka description repo (Panda hand)...")
    franka_repo = os.path.join(ASSETS_DIR, "franka_description")
    try:
        ok_franka = git_clone_if_needed("https://github.com/frankaemika/franka_description.git", franka_repo)
        if not ok_franka:
            failed_attempts.append(("Franka", "Git clone failed or repository not accessible"))
            print("  ❌ Failed: Git clone failed or repository not accessible")
        else:
            hand_candidates = glob.glob(os.path.join(franka_repo, "**", "visual", "*hand*.dae"), recursive=True)
            finger_candidates = glob.glob(os.path.join(franka_repo, "**", "visual", "*finger*.dae"), recursive=True)

            if not hand_candidates and not finger_candidates:
                failed_attempts.append(("Franka", "No hand/finger mesh files found in repository"))
                print("  ❌ Failed: No hand/finger mesh files found in repository")
            else:
                print(f"  Found {len(hand_candidates)} hand files, {len(finger_candidates)} finger files")
                meshes = []
                loaded_parts = []
                failed_files = []
                
                for p in (hand_candidates[:1] + finger_candidates[:2]):
                    try:
                        m = load_mesh_any(p)
                        if m is not None and len(m.vertices) > 0:
                            meshes.append(m)
                            loaded_parts.append(os.path.basename(p))
                        else:
                            failed_files.append((os.path.basename(p), "Empty mesh or no vertices"))
                    except Exception as e:
                        failed_files.append((os.path.basename(p), f"Load error: {str(e)[:80]}"))

                if meshes:
                    robot_mesh = normalize_mesh(trimesh.util.concatenate(meshes), target=0.9)
                    robot_mesh = maybe_decimate(robot_mesh, target_faces=3500)
                    robot_path = f"Franka: {'+'.join(loaded_parts)} (visual .dae)"
                    print(f"  ✓ Successfully loaded {len(meshes)} parts")
                    loaded_meshes.append((robot_mesh.copy(), "Franka Hand", robot_path))
                else:
                    reason = f"No valid mesh parts loaded. {len(failed_files)} files failed"
                    if failed_files:
                        reason += f" (e.g., {failed_files[0][0]}: {failed_files[0][1]})"
                    failed_attempts.append(("Franka", reason))
                    print(f"  ❌ Failed: {reason}")
    except Exception as e:
        failed_attempts.append(("Franka", f"Error: {str(e)[:100]}"))
        print(f"  ❌ Failed: {str(e)[:100]}")

# 3) MoveIt resources fallback (collision meshes are lighter)
if robot_mesh is None:
    print(f"\n[3/4] Trying MoveIt resources as fallback...")
    robot_repo = os.path.join(ASSETS_DIR, "moveit_resources")
    try:
        ok_moveit = git_clone_if_needed("https://github.com/moveit/moveit_resources.git", robot_repo)
        if not ok_moveit:
            failed_attempts.append(("MoveIt", "Git clone failed or repository not accessible"))
            print("  ❌ Failed: Git clone failed or repository not accessible")
        else:
            all_candidates = (
                glob.glob(os.path.join(robot_repo, "**", "*hand*.stl"), recursive=True) +
                glob.glob(os.path.join(robot_repo, "**", "*gripper*.stl"), recursive=True) +
                glob.glob(os.path.join(robot_repo, "**", "*hand*.dae"), recursive=True) +
                glob.glob(os.path.join(robot_repo, "**", "*gripper*.dae"), recursive=True)
            )

            if not all_candidates:
                failed_attempts.append(("MoveIt", "No hand/gripper mesh files found in repository"))
                print("  ❌ Failed: No hand/gripper mesh files found in repository")
            else:
                print(f"  Found {len(all_candidates)} candidate files, trying to load...")
                
                def score_mesh(path):
                    score = 0
                    pl = path.lower()
                    if "robotiq" in pl:
                        score += 2000
                    if "panda" in pl and ("gripper" in pl or "hand" in pl):
                        score += 1500
                    if "/visual/" in path:
                        score += 500
                    if pl.endswith('.dae'):
                        score += 100
                    try:
                        score += min(os.path.getsize(path)//1000, 500)
                    except Exception:
                        pass
                    return score

                candidates_sorted = sorted(all_candidates, key=score_mesh, reverse=True)
                failed_files = []
                
                for candidate in candidates_sorted[:12]:
                    try:
                        loaded = load_mesh_any(candidate)
                        if loaded is not None and len(loaded.vertices) > 0:
                            robot_mesh = normalize_mesh(loaded, target=0.9)
                            robot_mesh = maybe_decimate(robot_mesh, target_faces=2500)
                            robot_path = os.path.relpath(candidate, robot_repo)
                            print(f"  ✓ Successfully loaded: {os.path.basename(candidate)}")
                            loaded_meshes.append((robot_mesh.copy(), "MoveIt Mesh", robot_path))
                            break
                        else:
                            failed_files.append((os.path.basename(candidate), "Empty mesh or no vertices"))
                    except Exception as e:
                        failed_files.append((os.path.basename(candidate), f"Load error: {str(e)[:80]}"))
                        continue
                
                if robot_mesh is None:
                    reason = f"No valid mesh loaded. Tried {len(candidates_sorted[:12])} files, {len(failed_files)} failed"
                    if failed_files:
                        reason += f" (e.g., {failed_files[0][0]}: {failed_files[0][1]})"
                    failed_attempts.append(("MoveIt", reason))
                    print(f"  ❌ Failed: {reason}")
    except Exception as e:
        failed_attempts.append(("MoveIt", f"Error: {str(e)[:100]}"))
        print(f"  ❌ Failed: {str(e)[:100]}")

# 4) Procedural final fallback
if robot_mesh is None:
    print(f"\n[4/4] Falling back to procedural gripper mesh (simple 3-box design)")
    try:
        robot_mesh = make_simple_gripper()
        robot_path = "procedural"
        print("  ✓ Successfully created procedural gripper")
        loaded_meshes.append((robot_mesh.copy(), "Procedural Gripper", robot_path))
    except Exception as e:
        failed_attempts.append(("Procedural", f"Failed to create: {str(e)[:100]}"))
        print(f"  ❌ Failed: {str(e)[:100]}")
        raise RuntimeError("All mesh loading attempts failed, including procedural fallback!")

# Use this as the visualization mesh by default
robot_mesh_vis = robot_mesh

# Summary
print("\n" + "="*70)
print("MESH LOADING SUMMARY")
print("="*70)
print(f"\n✓ PRIMARY MESH (used in demos if only one loaded):")
print(f"   Name: {robot_path}")
print(f"   Stats: {len(robot_mesh.vertices)} vertices, {len(robot_mesh.faces)} faces")
print(f"\n📦 ALL SUCCESSFULLY LOADED MESHES ({len(loaded_meshes)} total):")
for i, (mesh, name, source) in enumerate(loaded_meshes, 1):
    status = "⭐ PRIMARY" if i == 1 and len(loaded_meshes) == 1 else f"   #{i}"
    print(f"   {status} {name}")
    print(f"      • Vertices: {len(mesh.vertices):,}, Faces: {len(mesh.faces):,}")
    print(f"      • Source: {source}")
    print()
print("="*70)
if failed_attempts:
    print(f"\n❌ FAILED ATTEMPTS ({len(failed_attempts)} total):")
    for source, reason in failed_attempts:
        print(f"   • {source}: {reason}")
    print()
else:
    print("\n✓ All attempted sources succeeded!")
print("="*70)
print("NOTE: Animated demos will be created for ALL loaded meshes below.")
print("      Each demo will clearly show which mesh it's using.")
if 'fast_simplification' not in globals():
    print("\n💡 Tip: If Plotly feels slow, run `pip install fast-simplification`")
print("="*70)


## Preview All Successfully Loaded Meshes

Static 3D previews of all meshes that were successfully loaded. Each mesh will also get its own animated demo below.


In [ ]:
# Visualize all successfully loaded meshes
if loaded_meshes:
    print(f"Visualizing {len(loaded_meshes)} successfully loaded mesh(es):\n")
    
    for i, (mesh, name, source) in enumerate(loaded_meshes, 1):
        print(f"  {i}. {name}")
        print(f"     Vertices: {len(mesh.vertices)}, Faces: {len(mesh.faces)}")
        print(f"     Source: {source}\n")
        
        # Create a simple static visualization
        fig = go.Figure()
        fig.add_trace(mesh_to_plotly(mesh, name=name, opacity=0.9))
        fig.add_traces(axes_traces(np.eye(3), L=1.0))
        set_scene(fig, lim=1.5, title=f"Loaded Mesh {i}: {name}")
        fig.show()
        print(f"✓ Displayed: {name}\n")
else:
    print("No meshes were successfully loaded.")


## Demo 1 — Basic 3D: tangential velocity \(\dot p = \omega \times p\)

In [ ]:
print("="*60)
print("Demo 1: Basic 3D - tangential velocity v = ω × p")
print("Note: ω is in SPACE frame (constant vector)")
print("="*60)

omega = np.array([0.2, 0.6, 1.0])   # rad/s (space frame)
p0 = unit([1.0, 0.3, 0.0])
print(f"Angular velocity ω (space): {omega} rad/s")
print(f"Initial point p0: {p0}")

T = 4.0
N = 80
ts = np.linspace(0, T, N)
print(f"Generating {N} animation frames...")

frames = []
for t in ts:
    R = rodrigues(omega, np.linalg.norm(omega)*t)
    p_t = R @ p0
    v_t = np.cross(omega, p_t)
    v_vis = 0.5 * v_t/(np.linalg.norm(v_t)+1e-12)

    data = []
    data += axes_traces(np.eye(3), L=1.2)
    data.append(arrow_trace(omega, name="ω (space frame)", scale=0.5))
    data.append(arrow_trace(p_t, name="p(t)", scale=1.0, width=12))
    data.append(arrow_trace(v_vis, origin=p_t, name="tangent v", scale=1.0, width=8))
    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

print(f"✓ Generated {len(frames)} frames. Creating visualization...")
fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.6, title="Demo 1: p(t) and tangent v = ω_space × p")
print("✓ Visualization ready!")

# Save animation
save_animation(fig, "demo1_basic_tangential_velocity", assets_dir=ASSETS_DIR, format="html")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 40, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 2 — Airplane: yaw/pitch/roll + wing-tip velocity
We estimate \(\omega\) from \(R(t)\) using \(\omega^\wedge \approx \dot R R^\top\), then draw \(v=\omega\times r\).

In [ ]:
print("="*60)
print("Demo 2: Airplane - yaw/pitch/roll + wing-tip velocity")
print(f"Using mesh: Procedural airplane mesh")
print(f"Mesh stats: {len(air_mesh.vertices)} vertices, {len(air_mesh.faces)} faces")
print("="*60)

V = air_mesh.vertices
wing_tip = V[np.argmax(V[:,1])]
print(f"Wing tip point (farthest +y): {wing_tip}")

T = 6.0
N = 90
ts = np.linspace(0, T, N)
print(f"Generating {N} animation frames...")

yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

Rs = []
for i in range(N):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
frames = []
for i,t in enumerate(ts):
    R = Rs[i]
    Vt = apply_R(R, air_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=air_mesh.faces, process=False)

    if 0 < i < N-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt

    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])  # ω in space frame
    omega_b = R.T @ omega_s  # ω in body frame (for reference)

    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)  # v = ω_space × r_space
    vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="airplane", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)   # space axes
    data += axes_traces(R, L=1.0)           # body axes
    data.append(arrow_trace(omega_s, name="ω (space frame)", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

print(f"✓ Generated {len(frames)} frames. Creating visualization...")
print("Note: ω is shown in SPACE frame (computed from Ṙ R^T)")
fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=2.0, title="Demo 2: Airplane yaw/pitch/roll + v = ω_space × r (wing tip)")
print("✓ Visualization ready!")

# Save animation
save_animation(fig, "demo2_airplane_yaw_pitch_roll", assets_dir=ASSETS_DIR, format="html")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 2b — Pretty Airplane: yaw/pitch/roll + wing-tip velocity (using the pretty mesh)



In [ ]:
import trimesh
import numpy as np

def create_triangular_prism(base_points_2d, height, center=[0, 0, 0]):
    """
    Create a triangular prism by extruding a 2D triangle along Z.
    base_points_2d: array of shape (3, 2) with triangle vertices in XY plane
    height: extrusion height along Z
    """
    base_points_2d = np.asarray(base_points_2d)
    # Create bottom and top faces
    bottom = np.column_stack([base_points_2d, np.zeros(3)])  # z=0
    top = np.column_stack([base_points_2d, np.full(3, height)])  # z=height
    
    # Vertices: bottom triangle (0,1,2) + top triangle (3,4,5)
    vertices = np.vstack([bottom, top])
    
    # Faces: 2 triangle faces (bottom, top) + 3 rectangular sides
    faces = np.array([
        # Bottom face
        [0, 2, 1],
        # Top face
        [3, 4, 5],
        # Side faces (rectangles as 2 triangles each)
        [0, 1, 3], [1, 4, 3],
        [1, 2, 4], [2, 5, 4],
        [2, 0, 5], [0, 3, 5],
    ], dtype=int)
    
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    if center != [0, 0, 0]:
        mesh.apply_translation(center)
    return mesh

def make_airplane_mesh_pretty():
    parts = []

    # Fuselage: capsule (rounded ends) looks nicer than a cylinder
    fus = trimesh.creation.capsule(radius=0.10, height=1.8, count=[24, 24])
    # capsule is along Z; rotate to X
    fus.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))
    parts.append(fus)

    # Nose cone
    nose = trimesh.creation.cone(radius=0.12, height=0.25, sections=32)
    nose.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [0,1,0]))
    nose.apply_translation([1.0, 0, 0])
    parts.append(nose)

    # Main wing: triangular prism
    wing_poly = np.array([[0.15, 0.0],
                          [-0.15, 0.55],
                          [-0.15, -0.55]])
    # Create prism with thin thickness in Z (0.02), then translate
    wing = create_triangular_prism(wing_poly, height=0.02)
    wing.apply_translation([0.1, 0, -0.01])  # center in Z
    parts.append(wing)

    # Tail wing (smaller triangular prism)
    tail_poly = np.array([[0.08, 0.0],
                          [-0.10, 0.25],
                          [-0.10, -0.25]])
    tail = create_triangular_prism(tail_poly, height=0.015)
    tail.apply_translation([-0.75, 0, 0.0125])  # center in Z
    parts.append(tail)

    # Vertical fin
    fin = trimesh.creation.box(extents=[0.20, 0.02, 0.22])
    fin.apply_translation([-0.78, 0, 0.14])
    parts.append(fin)

    # Propeller (simple cross)
    prop1 = trimesh.creation.box(extents=[0.02, 0.35, 0.01])
    prop2 = trimesh.creation.box(extents=[0.02, 0.35, 0.01])
    prop2.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2, [1,0,0]))
    prop = trimesh.util.concatenate([prop1, prop2])
    prop.apply_translation([1.05, 0, 0])
    parts.append(prop)

    plane = trimesh.util.concatenate(parts)
    return normalize_mesh(plane, target=1.0)

air_mesh = make_airplane_mesh_pretty()


In [ ]:
print("="*60)
print("Demo 2b: Pretty Airplane - yaw/pitch/roll + wing-tip velocity")
print(f"Using mesh: Procedural airplane mesh (pretty version)")
print(f"Mesh stats: {len(air_mesh.vertices)} vertices, {len(air_mesh.faces)} faces")
print("="*60)

V = air_mesh.vertices
wing_tip = V[np.argmax(V[:,1])]
print(f"Wing tip point (farthest +y): {wing_tip}")

T = 6.0
N = 90
ts = np.linspace(0, T, N)
print(f"Generating {N} animation frames...")

yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

Rs = []
for i in range(N):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
frames = []
for i,t in enumerate(ts):
    R = Rs[i]
    Vt = apply_R(R, air_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=air_mesh.faces, process=False)

    if 0 < i < N-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt

    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])  # ω in space frame
    omega_b = R.T @ omega_s  # ω in body frame (for reference)

    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)  # v = ω_space × r_space
    vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="airplane", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)   # space axes
    data += axes_traces(R, L=1.0)           # body axes
    data.append(arrow_trace(omega_s, name="ω (space frame)", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

print(f"✓ Generated {len(frames)} frames. Creating visualization...")
print("Note: ω is shown in SPACE frame (computed from Ṙ R^T)")
fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=2.0, title="Demo 2b: Pretty Airplane yaw/pitch/roll + v = ω_space × r (wing tip)")
print("✓ Visualization ready!")

# Save animation
save_animation(fig, "demo2b_pretty_airplane_yaw_pitch_roll", assets_dir=ASSETS_DIR, format="html")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig



airplane 3d transformation video

## Load a real airplane mesh (Flightradar24) if possible; otherwise fallback



In [ ]:
aircraft_repo = os.path.join(ASSETS_DIR, "fr24-3d-models")
print("Attempting to load Flightradar24 aircraft models...")
print(f"Repository path: {aircraft_repo}")

ok_aircraft = git_clone_if_needed("https://github.com/millimoose/fr24-3d-models.git", aircraft_repo)

aircraft_mesh_real = None
gltf1_count = 0
other_errors = []

if ok_aircraft:
    print("✓ Repository cloned/accessed successfully")
    # Search for glTF2 files (.glb and .gltf)
    aircraft_candidates = glob.glob(os.path.join(aircraft_repo, "**", "*.glb"), recursive=True) + \
                         glob.glob(os.path.join(aircraft_repo, "**", "*.gltf"), recursive=True)
    
    print(f"Found {len(aircraft_candidates)} candidate model files")
    
    if aircraft_candidates:
        # Try each candidate until one works (some might be glTF1)
        for i, candidate_path in enumerate(sorted(aircraft_candidates, key=lambda p: os.path.getsize(p)), 1):
            try:
                # Only print first few attempts to avoid spam, then summarize
                if i <= 5:
                    print(f"Trying [{i}/{len(aircraft_candidates)}]: {os.path.basename(candidate_path)}")
                loaded = load_mesh_any(candidate_path)
                if loaded is not None and len(loaded.vertices) > 0:
                    aircraft_mesh_real = normalize_mesh(loaded, target=1.0)
                    print(f"\n✓ Successfully loaded: {candidate_path}")
                    print(f"  Mesh: {len(aircraft_mesh_real.vertices)} vertices, {len(aircraft_mesh_real.faces)} faces")
                    break
            except Exception as e:
                error_msg = str(e)
                if "GLTF" in error_msg and ("not supported" in error_msg or "not `1" in error_msg or "not `1.0" in error_msg):
                    gltf1_count += 1
                    if i <= 5:
                        print(f"  ⚠ glTF 1.0 detected (trimesh only supports glTF 2.0)")
                else:
                    other_errors.append((os.path.basename(candidate_path), error_msg))
                    if i <= 5:
                        print(f"  ⚠ Error: {error_msg}")
                continue
    else:
        print("⚠ No model files found in repository")
else:
    print("⚠ Failed to clone/access repository (git may not be available or network issue)")

# Summary
print("\n" + "="*60)
if aircraft_mesh_real is None:
    print("❌ No compatible aircraft mesh loaded")
    print("\nFailure summary:")
    if not ok_aircraft:
        print("  - Git clone/access failed (check git availability and network)")
    elif not aircraft_candidates:
        print("  - No model files found in repository")
    else:
        if gltf1_count > 0:
            print(f"  - {gltf1_count}/{len(aircraft_candidates)} models are glTF 1.0 (not supported by trimesh)")
        if other_errors:
            print(f"  - {len(other_errors)} models failed with other errors:")
            for name, err in other_errors[:3]:  # Show first 3
                print(f"    • {name}: {err}")
            if len(other_errors) > 3:
                print(f"    ... and {len(other_errors) - 3} more")
    print("\n✓ Falling back to procedural airplane mesh (pretty version)")
else:
    print("✓ Successfully loaded real aircraft mesh from Flightradar24")

aircraft_mesh_real = aircraft_mesh_real if aircraft_mesh_real is not None else make_airplane_mesh_pretty()
print(f"\nFinal mesh: {len(aircraft_mesh_real.vertices)} vertices, {len(aircraft_mesh_real.faces)} faces")
print("="*60)



## Demo 3 — Real Aircraft Mesh: yaw/pitch/roll + wing-tip velocity

Using a real airplane mesh from Flightradar24 if available, otherwise falls back to procedural mesh.



In [ ]:
print("Demo 3: Starting visualization generation...")
V = aircraft_mesh_real.vertices
wing_tip = V[np.argmax(V[:,1])]

T = 6.0
N = 90
ts = np.linspace(0, T, N)
print(f"Generating {N} animation frames...")

yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

Rs = []
for i in range(N):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
frames = []
for i,t in enumerate(ts):
    R = Rs[i]
    Vt = apply_R(R, aircraft_mesh_real.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=aircraft_mesh_real.faces, process=False)

    if 0 < i < N-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt

    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])

    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)
    vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="aircraft", opacity=0.95))
    data += axes_traces(np.eye(3), L=1.2)   # space axes
    data += axes_traces(R, L=1.0)           # body axes
    data.append(arrow_trace(omega_s, name="ω", scale=0.45, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

print(f"Frames generated ({len(frames)} total). Creating Plotly figure...")
fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=2.0, title="Demo 3: Real Aircraft yaw/pitch/roll + v = ω × r (wing tip)")

fig.update_layout(
    updatemenus=[dict(type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
print("Demo 3: Figure created successfully! Rendering...")
fig
# print('Done!')



In [ ]:
print("Cell 20 executed - Demo 3 completed successfully")
print("This confirms execution can continue after Demo 3")

In [ ]:
print('hi')

## Demo 4 — Robot mesh: spin + fingertip velocity

In [ ]:
print('hi')

In [ ]:
# Prepare all loaded meshes for animation (simplify if needed)
print("="*70)
print("PREPARING ALL LOADED MESHES FOR ANIMATION")
print("="*70)

TARGET_FACES = 10000  # Target face count for animation (balance between quality and speed)

# Prepare visualization versions of all meshes
prepared_meshes = []  # List of (mesh_vis, name, source, original_stats) tuples

for i, (mesh, name, source) in enumerate(loaded_meshes, 1):
    print(f"\n[{i}/{len(loaded_meshes)}] Preparing: {name}")
    print(f"   Original: {len(mesh.vertices):,} vertices, {len(mesh.faces):,} faces")
    
    mesh_vis = mesh.copy()
    
    if len(mesh_vis.faces) <= TARGET_FACES:
        print(f"   ✓ Already <= {TARGET_FACES} faces; no simplification needed")
    else:
        try:
            import fast_simplification  # noqa: F401
            mesh_vis = mesh_vis.simplify_quadric_decimation(face_count=TARGET_FACES)
            print(f"   ✓ Simplified to: {len(mesh_vis.vertices):,} vertices, {len(mesh_vis.faces):,} faces")
        except Exception as e:
            print(f"   ⚠ Simplification not available (or failed): {e}")
            print(f"   → Using original mesh (may be slow for large meshes)")
            print(f"   💡 Tip: pip install fast-simplification for better performance")
    
    original_stats = (len(mesh.vertices), len(mesh.faces))
    prepared_meshes.append((mesh_vis, name, source, original_stats))
    print(f"   ✓ Ready for animation")

print("\n" + "="*70)
print(f"✓ Prepared {len(prepared_meshes)} mesh(es) for animation")
print("="*70)


## Animated Demos for ALL Loaded Robot Meshes

Each successfully loaded mesh gets its own animated demo showing:
- **Demo A**: Simple spin with constant angular velocity
- **Demo B**: Yaw/pitch/roll motion (like airplane demo) with estimated ω from R(t)

Each demo clearly labels which mesh it's using.


In [ ]:
# Create animated demos for ALL successfully loaded meshes
if not prepared_meshes:
    print("⚠ No meshes prepared for animation. Run the previous cell first.")
else:
    print("="*70)
    print(f"CREATING ANIMATED DEMOS FOR {len(prepared_meshes)} MESH(ES)")
    print("="*70)
    
    for mesh_idx, (mesh_vis, name, source, (orig_v, orig_f)) in enumerate(prepared_meshes, 1):
        print(f"\n{'='*70}")
        print(f"MESH #{mesh_idx}/{len(prepared_meshes)}: {name}")
        print(f"{'='*70}")
        print(f"Source: {source}")
        print(f"Stats: {len(mesh_vis.vertices):,} vertices, {len(mesh_vis.faces):,} faces (for animation)")
        if orig_f > len(mesh_vis.faces):
            print(f"Original: {orig_v:,} vertices, {orig_f:,} faces (simplified for performance)")
        
        V = mesh_vis.vertices
        fingertip = V[np.argmax(V[:,0])]  # Farthest point in +x direction
        print(f"Fingertip point (farthest +x): {fingertip}")
        
        # ===== DEMO A: Simple Spin =====
        print(f"\n--- Demo A: Simple Spin ---")
        omega = np.array([0.0, 1.2, 0.6])  # rad/s (space frame)
        print(f"Angular velocity ω (space): {omega} rad/s")
        
        T = 4.0
        N = 80
        ts = np.linspace(0, T, N)
        print(f"Generating {N} animation frames...")
        
        frames = []
        for t in ts:
            theta = np.linalg.norm(omega)*t
            R = rodrigues(omega, theta)
            
            Vt = apply_R(R, mesh_vis.vertices)
            mesh_t = trimesh.Trimesh(vertices=Vt, faces=mesh_vis.faces, process=False)
            
            pt = R @ fingertip
            vt = np.cross(omega, pt)
            vt_vis = 0.45 * vt/(np.linalg.norm(vt)+1e-12)
            
            data = []
            data.append(mesh_to_plotly(mesh_t, name=name, opacity=0.95))
            data += axes_traces(np.eye(3), L=1.2)   # space axes
            data += axes_traces(R, L=1.0)           # body axes
            data.append(arrow_trace(omega, name="ω (space frame)", scale=0.45, width=10))
            data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
            data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))
            
            frames.append(go.Frame(data=data, name=f"{t:.2f}"))
        
        print(f"✓ Generated {len(frames)} frames. Creating visualization...")
        fig = go.Figure(frames=frames)
        fig.add_traces(frames[0].data)
        set_scene(fig, lim=1.8, title=f"Demo A: {name} - Spin + v = ω_space × r (fingertip)")
        
        fig.update_layout(
            updatemenus=[dict(type="buttons",
                buttons=[
                    dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
                    dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
                ],
                x=0.02, y=0.02
            )],
            sliders=[dict(
                steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
                x=0.1, y=0.02, len=0.85
            )]
        )
        print("✓ Visualization ready!")
        
        # Save animation
        mesh_name_safe = name.replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_")[:30]
        save_animation(fig, f"demoA_spin_{mesh_name_safe}", assets_dir=ASSETS_DIR, format="html")
        fig
        
        # ===== DEMO B: Yaw/Pitch/Roll =====
        print(f"\n--- Demo B: Yaw/Pitch/Roll ---")
        T = 6.0
        N = 90
        ts = np.linspace(0, T, N)
        print(f"Generating {N} animation frames with yaw/pitch/roll motion...")
        
        # Create yaw/pitch/roll motion
        yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
        pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
        roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)
        
        # Compute rotation matrices for each time step
        Rs = []
        for i in range(N):
            R = (rodrigues([0,0,1], yaw[i]) @
                 rodrigues([0,1,0], pitch[i]) @
                 rodrigues([1,0,0], roll[i]))
            Rs.append(R)
        Rs = np.array(Rs)
        
        # Estimate angular velocity from rotation matrices
        dt = ts[1]-ts[0]
        frames = []
        for i,t in enumerate(ts):
            R = Rs[i]
            Vt = apply_R(R, mesh_vis.vertices)
            mesh_t = trimesh.Trimesh(vertices=Vt, faces=mesh_vis.faces, process=False)
            
            # Numerical derivative of R to estimate omega
            if 0 < i < N-1:
                Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
            elif i == 0:
                Rdot = (Rs[1]-Rs[0])/dt
            else:
                Rdot = (Rs[-1]-Rs[-2])/dt
            
            # omega^hat = Rdot @ R^T, then extract omega vector
            omega_hat = Rdot @ R.T
            omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])  # ω in space frame
            
            # Compute fingertip position and velocity
            pt = R @ fingertip
            vt = np.cross(omega_s, pt)  # v = ω_space × r_space
            vt_vis = 0.5 * vt/(np.linalg.norm(vt)+1e-12)
            
            # Build visualization data
            data = []
            data.append(mesh_to_plotly(mesh_t, name=name, opacity=0.95))
            data += axes_traces(np.eye(3), L=1.2)   # space axes
            data += axes_traces(R, L=1.0)           # body axes
            data.append(arrow_trace(omega_s, name="ω (space frame)", scale=0.45, width=10))
            data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
            data.append(arrow_trace(vt_vis, origin=pt, name="v", scale=1.0, width=8))
            
            frames.append(go.Frame(data=data, name=f"{t:.2f}"))
        
        print(f"✓ Generated {len(frames)} frames. Creating visualization...")
        print("Note: ω is shown in SPACE frame (computed from Ṙ R^T)")
        fig = go.Figure(frames=frames)
        fig.add_traces(frames[0].data)
        set_scene(fig, lim=1.8, title=f"Demo B: {name} - Yaw/Pitch/Roll + v = ω_space × r (fingertip)")
        
        fig.update_layout(
            updatemenus=[dict(type="buttons",
                buttons=[
                    dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
                    dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
                ],
                x=0.02, y=0.02
            )],
            sliders=[dict(
                steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
                x=0.1, y=0.02, len=0.85
            )]
        )
        print("✓ Visualization ready!")
        
        # Save animation
        save_animation(fig, f"demoB_yaw_pitch_roll_{mesh_name_safe}", assets_dir=ASSETS_DIR, format="html")
        fig
        
        print(f"\n✓ Completed demos for {name}")
    
    print(f"\n{'='*70}")
    print(f"✓ ALL DEMOS COMPLETE! Created animations for {len(prepared_meshes)} mesh(es)")
    print(f"  Saved to: {ASSETS_DIR}/")
    print("="*70)


In [ ]:
# # Set Plotly to use notebook renderer (works in Jupyter/WSL, avoids browser requirement)
# import plotly.io as pio
# pio.renderers.default = "notebook"
# import sys
# sys.exit

In [ ]:
# This cell has been replaced by the comprehensive demo loop above.
# All meshes now get their own animated demos (Demo A and Demo B).
# See the "Animated Demos for ALL Loaded Robot Meshes" section above.
print("✓ This old single-mesh demo has been replaced.")
print("  All loaded meshes now get animated demos in the section above.")


In [ ]:
print('hi')

# Another Robot

In [ ]:
import os, glob, numpy as np, trimesh

# --- Pick an industrial gripper repo (Robotiq 2F-85) ---
# If git_clone_if_needed already exists in your notebook, this will reuse it.
robotiq_repo = os.path.join(ASSETS_DIR, "robotiq")
ok_robotiq = git_clone_if_needed(
    "https://github.com/ros-industrial/robotiq.git",
    robotiq_repo,
    depth=1
)

def _load_and_concat(paths):
    meshes = []
    for p in paths:
        try:
            m = load_mesh_any(p)
            if m is not None and len(m.vertices) > 0:
                meshes.append(m)
        except Exception as e:
            print("  ⚠ load failed:", os.path.basename(p), str(e)[:80])
    if not meshes:
        return None
    return trimesh.util.concatenate(meshes)

robot_mesh = None
robot_path = None

if ok_robotiq:
    print("✓ Robotiq repo available:", robotiq_repo)

    # Look for 2F-85 (and 2F-140 as fallback) visual meshes.
    # Repo layout varies; this finds common patterns.
    stl_candidates = glob.glob(os.path.join(robotiq_repo, "**", "*.stl"), recursive=True)
    dae_candidates = glob.glob(os.path.join(robotiq_repo, "**", "*.dae"), recursive=True)

    # Prefer STL (no pycollada). Prefer "visual" and "2f_85".
    def score(p):
        pl = p.lower()
        s = 0
        if "2f_85" in pl or "2f85" in pl: s += 2000
        if "gripper" in pl: s += 400
        if "visual" in pl: s += 300
        if pl.endswith(".stl"): s += 200
        if "collision" in pl: s -= 100
        # Larger files tend to be more detailed (light heuristic)
        try:
            s += min(os.path.getsize(p)//5000, 200)
        except:
            pass
        return s

    candidates_sorted = sorted(stl_candidates + dae_candidates, key=score, reverse=True)

    # Heuristic: grab top N files that look like they're part of the gripper
    top = []
    for p in candidates_sorted:
        pl = p.lower()
        if ("2f_85" in pl or "2f85" in pl or "2f-85" in pl) and ("finger" in pl or "base" in pl or "gripper" in pl or "coupler" in pl):
            top.append(p)
        if len(top) >= 12:
            break

    # If we didn’t catch enough, broaden to “robotiq gripper” parts
    if len(top) < 3:
        for p in candidates_sorted:
            pl = p.lower()
            if ("robotiq" in pl and "gripper" in pl) and ("finger" in pl or "base" in pl or "coupler" in pl):
                top.append(p)
            if len(top) >= 12:
                break

    print(f"  Selected {len(top)} candidate part mesh files")
    for p in top[:8]:
        print("   -", os.path.relpath(p, robotiq_repo))

    robot_mesh = _load_and_concat(top)

    if robot_mesh is not None:
        # Center + scale for consistent viewing
        robot_mesh = robot_mesh.copy()
        robot_mesh.apply_translation(-robot_mesh.centroid)

        # Normalize to ~unit-ish size (your normalize_mesh expects something like this)
        robot_mesh = normalize_mesh(robot_mesh, target=0.9)

        robot_path = "Robotiq 2F-85 (assembled from repo meshes)"
        print("✓ Loaded industrial gripper mesh!")
        print("  verts:", len(robot_mesh.vertices), "faces:", len(robot_mesh.faces))
    else:
        print("⚠ Could not assemble Robotiq mesh; falling back.")
        robot_mesh = make_simple_gripper()
        robot_path = "procedural"
else:
    print("⚠ Robotiq repo clone failed; falling back to procedural.")
    robot_mesh = make_simple_gripper()
    robot_path = "procedural"

# Use this as the visualization mesh by default
robot_mesh_vis = robot_mesh
print("Robot mesh source:", robot_path)
